# Importing Libraries

In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from sklearn.linear_model import LogisticRegression

# List of Classifiers

In [2]:
classifiers = [
    LogisticRegression()
    
]

# Create a list to store results for each classifier
results = []

# Define the number of folds for cross-validation

In [3]:
n_splits = 5

In [4]:
# Create StratifiedKFold cross-validator
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Loading Dataset

In [5]:
# Define a function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    labels = []
    for subfolder_name in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder_name)
        if os.path.isdir(subfolder_path):
            label = subfolder_name  # Assuming folder names are class labels
            for filename in os.listdir(subfolder_path):
                img = cv2.imread(os.path.join(subfolder_path, filename))
                if img is not None:
                    img = cv2.resize(img, (224, 224))  # Adjust the size as needed
                    images.append(img)
                    labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess your images
X, y = load_images_from_folder('Dataset')

# Data preprocessing: Normalize pixel values to the range [0, 1]

In [6]:
X = X / 255.0

# Make Data 1D for compatability with Standard Classifiers

In [7]:
X_flat = X.reshape(X.shape[0], -1)

# Create a DataFrame to store results

In [8]:
results_df = pd.DataFrame(columns=['Classifier', 'Fold', 'Precision', 'Recall', 'F1 Score', 'Accuracy'])

# Perform cross-validation for each classifier

In [9]:
import time

results = []

for clf in classifiers:
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []
    fold_times = []  # Store fold execution times
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_flat, y), 1):
        X_train, X_val = X_flat[train_index], X_flat[val_index]
        y_train, y_val = y[train_index], y[val_index]

        start_time = time.time()  # Record start time
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)

        precision = precision_score(y_val, y_pred, average='weighted')
        recall = recall_score(y_val, y_pred, average='weighted')
        f1 = f1_score(y_val, y_pred, average='weighted')
        accuracy = accuracy_score(y_val, y_pred)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        accuracy_scores.append(accuracy)
        
        end_time = time.time()  # Record end time
        fold_time = end_time - start_time  # Calculate fold execution time
        fold_times.append(fold_time)

        results_df = results_df.append({
            'Classifier': clf.__class__.__name__,
            'Fold': fold,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'Accuracy': accuracy,
            'Fold Time (s)': fold_time  # Store fold execution time
        }, ignore_index=True)

    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracy_scores)
    avg_fold_time = np.mean(fold_times)  # Calculate average fold execution time

    results.append({
        'Classifier': clf.__class__.__name__,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1 Score': avg_f1,
        'Accuracy': avg_accuracy,
        'Average Fold Time (s)': avg_fold_time  # Store average fold execution time
    })

# Print the results for each fold and the average result along with fold execution time
for index, row in results_df.iterrows():
    print(f"Classifier: {row['Classifier']}, Fold: {row['Fold']}")
    print(f"Precision: {row['Precision']:.4f}")
    print(f"Recall: {row['Recall']:.4f}")
    print(f"F1 Score: {row['F1 Score']:.4f}")
    print(f"Accuracy: {row['Accuracy']:.4f}")
    print(f"Fold Execution Time: {row['Fold Time (s)']:.4f} seconds")  # Print fold execution time
    print()

# Print the average results along with average fold execution time
for result in results:
    print(f"Classifier: {result['Classifier']}")
    print(f"Average Precision: {result['Precision']:.4f}")
    print(f"Average Recall: {result['Recall']:.4f}")
    print(f"Average F1 Score: {result['F1 Score']:.4f}")
    print(f"Average Accuracy: {result['Accuracy']:.4f}")
    print(f"Average Fold Execution Time: {result['Average Fold Time (s)']:.4f} seconds")  # Print average fold execution time
    print()

# Save results to an Excel file
results_df.to_excel('Cross Validation ML.xlsx', index=False)


C:\Users\nafem\anaconda3\envs\gpu\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\nafem\AppData\Local\Temp\ipykernel_1476\4154691716.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\nafem\anaconda3\envs\gpu\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or 

Classifier: LogisticRegression, Fold: 1
Precision: 0.9955
Recall: 0.9955
F1 Score: 0.9954
Accuracy: 0.9955
Fold Execution Time: 21.1267 seconds

Classifier: LogisticRegression, Fold: 2
Precision: 0.9879
Recall: 0.9864
F1 Score: 0.9866
Accuracy: 0.9864
Fold Execution Time: 24.2669 seconds

Classifier: LogisticRegression, Fold: 3
Precision: 0.9911
Recall: 0.9909
F1 Score: 0.9907
Accuracy: 0.9909
Fold Execution Time: 24.7978 seconds

Classifier: LogisticRegression, Fold: 4
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Accuracy: 1.0000
Fold Execution Time: 23.8928 seconds

Classifier: LogisticRegression, Fold: 5
Precision: 0.9955
Recall: 0.9954
F1 Score: 0.9954
Accuracy: 0.9954
Fold Execution Time: 24.4986 seconds

Classifier: LogisticRegression
Average Precision: 0.9940
Average Recall: 0.9936
Average F1 Score: 0.9936
Average Accuracy: 0.9936
Average Fold Execution Time: 23.7166 seconds

